# Wrappers example

We will see on this notebook what are the wrappers defined by Sinergym and how to use them. You can also implement your own wrappers inheriting from *gym.Wrapper* or some of its variants.

In [1]:
import gymnasium as gym
import numpy as np
import sinergym
from sinergym.utils.wrappers import *
from sinergym.utils.constants import RANGES_5ZONE

## Multi-Objective Wrapper

[MO-Gymnasium](https://github.com/Farama-Foundation/MO-Gymnasium) is an open source Python library for developing and comparing multi-objective reinforcement learning algorithms. These environments return a reward vector instead of a scalar value; one for each objective.

In order to be more general as possible, it could be interesting that Sinergym would give that reward vector too. In this way, Sinergym would have compatibility with both; multi-objective algorithms and algorithms that work with a traditional reward value.

We can transform reward returned in a vector using the next wrapper:

In [2]:
env=gym.make('Eplus-5Zone-hot-discrete-v1')
env=MultiObjectiveReward(env,reward_terms=['reward_energy','reward_comfort'])

[2023-05-26 08:51:52,324] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model ExternalInterface object if it is not present...
[2023-05-26 08:51:52,325] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2023-05-26 08:51:52,326] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating building model OutPut:Variable and variables XML tree model for BVCTB connection.
[2023-05-26 08:51:52,327] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Setting up extra configuration in building model if exists...
[2023-05-26 08:51:52,327] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Setting up action definition in building model if exists...


You have to ensure that `reward_terms` are available in `info` dict returned in step method of the environment. Otherwise, we will encounter an execution error.
By default, Sinergym environments return in `info` dict all reward terms specified in reward class used, so if the objective exists in reward term you shouldn't have any problem.

In [3]:
env.reset()
action = env.action_space.sample()
obs, reward, terminated, truncated, info = env.step(action)
env.close()
print(reward)

[2023-05-26 08:51:52,373] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-05-26 08:51:52,492] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/examples/Eplus-env-5Zone-hot-discrete-v1-res1/Eplus-env-sub_run1


/usr/local/lib/python3.10/dist-packages/opyplus/weather_data/weather_data.py:493: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  epw_content = self._headers_to_epw(use_datetimes=use_datetimes) + df.to_csv(


[2023-05-26 08:51:59,208] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus simulation closed successfully. 
[-0.08284143205459649, -0.30864953312002896]


## Previous Observations Wrappers

This Wrapper will add observation values from previous timestep to current environment observation.
It is possible to select the variables you want to track its previous observation values. 
Observation space will be updated with the new dimension.

In [4]:
env=gym.make('Eplus-5Zone-hot-discrete-v1')
env = PreviousObservationWrapper(env, previous_variables=[
        'Zone Thermostat Heating Setpoint Temperature(SPACE1-1)',
        'Zone Thermostat Cooling Setpoint Temperature(SPACE1-1)',
        'Zone Air Temperature(SPACE1-1)'])

[2023-05-26 08:51:59,325] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model ExternalInterface object if it is not present...
[2023-05-26 08:51:59,325] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model ExternalInterface object if it is not present...
[2023-05-26 08:51:59,327] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2023-05-26 08:51:59,327] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2023-05-26 08:51:59,328] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating building model OutPut:Variable and variables XML tree model for BVCTB connection.
[2023-05-26 08:51:59,328] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating building model OutPut:Variable and variables XML tree model for BVCTB connection.
[2023-05

We can see the observation values has been updated:

In [5]:
env.reset()
obs,_,_,_,_=env.step(env.action_space.sample())
obs_dict=dict(zip(env.variables['observation'],obs))
env.close()
print('NEW OBSERVATION: ',obs_dict)

[2023-05-26 08:51:59,414] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-05-26 08:51:59,414] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-05-26 08:51:59,529] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/examples/Eplus-env-5Zone-hot-discrete-v1-res2/Eplus-env-sub_run1
[2023-05-26 08:51:59,529] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/examples/Eplus-env-5Zone-hot-discrete-v1-res2/Eplus-env-sub_run1


/usr/local/lib/python3.10/dist-packages/opyplus/weather_data/weather_data.py:493: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  epw_content = self._headers_to_epw(use_datetimes=use_datetimes) + df.to_csv(


[2023-05-26 08:52:05,937] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus simulation closed successfully. 
[2023-05-26 08:52:05,937] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus simulation closed successfully. 
NEW OBSERVATION:  {'year': 1991.0, 'month': 1.0, 'day': 1.0, 'hour': 0.0, 'Site Outdoor Air Drybulb Temperature(Environment)': 4.4, 'Site Outdoor Air Relative Humidity(Environment)': 65.0, 'Site Wind Speed(Environment)': 3.875, 'Site Wind Direction(Environment)': 145.0, 'Site Diffuse Solar Radiation Rate per Area(Environment)': 0.0, 'Site Direct Solar Radiation Rate per Area(Environment)': 0.0, 'Zone Thermostat Heating Setpoint Temperature(SPACE1-1)': 21.0, 'Zone Thermostat Cooling Setpoint Temperature(SPACE1-1)': 24.0, 'Zone Air Temperature(SPACE1-1)': 21.00007, 'Zone Thermal Comfort Mean Radiant Temperature(SPACE1-1 PEOPLE 1)': 19.689066, 'Zone Air Relative Humidity(SPACE1-1)': 23.126598, 'Zone Thermal Comfort Clothing Value(SPACE1-1 PEOPLE 1

## Datetime Wrapper

This wrapper will substitute `day` value by `is_weekend` flag, and `hour` and `month` by sin and cos values. 
Observation space is also updated automatically.

In [6]:
env=gym.make('Eplus-5Zone-hot-discrete-v1')
env = DatetimeWrapper(env)

[2023-05-26 08:52:06,066] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model ExternalInterface object if it is not present...
[2023-05-26 08:52:06,066] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model ExternalInterface object if it is not present...
[2023-05-26 08:52:06,066] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model ExternalInterface object if it is not present...
[2023-05-26 08:52:06,068] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2023-05-26 08:52:06,068] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2023-05-26 08:52:06,068] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2023-05-26 08:52:06,0

Concretely, this wrapper deletes the observation variables `month`, `day` and `hour`, and adds `month_sin`, `month_cos`, `is_weekend`, `hour_sin` and `hour_cos` instead:

In [7]:
env.reset()
obs,_,_,_,_=env.step(env.action_space.sample())
obs_dict=dict(zip(env.variables['observation'],obs))
env.close()
print('NEW OBSERVATION: ',obs_dict)

[2023-05-26 08:52:06,146] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-05-26 08:52:06,146] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-05-26 08:52:06,146] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-05-26 08:52:06,259] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/examples/Eplus-env-5Zone-hot-discrete-v1-res3/Eplus-env-sub_run1
[2023-05-26 08:52:06,259] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/examples/Eplus-env-5Zone-hot-discrete-v1-res3/Eplus-env-sub_run1
[2023-05-26 08:52:06,259] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/examples/Eplus-env-5Zone-hot-discrete-v1-res3/Eplus-env-sub_run1


/usr/local/lib/python3.10/dist-packages/opyplus/weather_data/weather_data.py:493: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  epw_content = self._headers_to_epw(use_datetimes=use_datetimes) + df.to_csv(


[2023-05-26 08:52:12,743] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus simulation closed successfully. 
[2023-05-26 08:52:12,743] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus simulation closed successfully. 
[2023-05-26 08:52:12,743] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus simulation closed successfully. 
NEW OBSERVATION:  {'year': 1991.0, 'month_cos': 1.0, 'month_sin': 0.0, 'is_weekend': 0.0, 'hour_cos': 1.0, 'hour_sin': 0.0, 'Site Outdoor Air Drybulb Temperature(Environment)': 4.400000095367432, 'Site Outdoor Air Relative Humidity(Environment)': 65.0, 'Site Wind Speed(Environment)': 3.875, 'Site Wind Direction(Environment)': 145.0, 'Site Diffuse Solar Radiation Rate per Area(Environment)': 0.0, 'Site Direct Solar Radiation Rate per Area(Environment)': 0.0, 'Zone Thermostat Heating Setpoint Temperature(SPACE1-1)': 22.0, 'Zone Thermostat Cooling Setpoint Temperature(SPACE1-1)': 23.0, 'Zone Air Temperature(SPACE1-1)': 21.3995

## Discrete Incremental Wrapper

A wrapper for an incremental discrete setpoint action space environment. This wrapper
will update environment action mapping and action space depending on the step and delta specified. The action will be sum
with current setpoint values instead of overwrite the latest action.
A discrete environment with only temperature setpoints control must be used with this wrapper.

In [8]:
env=gym.make('Eplus-5Zone-hot-discrete-v1')
print('ORIGINAL ACTION SPACE: ',env.action_space)
print('ORIGINAL ACTION MAPPING: ',env.action_mapping)
env = DiscreteIncrementalWrapper(
        env, max_values=[
            22.5, 30.0], min_values=[
            15.0, 22.5], delta_temp=2, step_temp=0.5)
print('WRAPPED ACTION SPACE: ',env.action_space)
print('WRAPPED ACTION MAPPING: ',env.action_mapping)

[2023-05-26 08:52:12,868] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model ExternalInterface object if it is not present...
[2023-05-26 08:52:12,868] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model ExternalInterface object if it is not present...
[2023-05-26 08:52:12,868] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model ExternalInterface object if it is not present...
[2023-05-26 08:52:12,868] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model ExternalInterface object if it is not present...
[2023-05-26 08:52:12,870] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2023-05-26 08:52:12,870] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2023-05-26 08:52:12,870] EPLUS_ENV_5Zone-ho

maximum and minimum values are specified so that the setpoint increments and decrements do not go outside the agreed limits.
The delta and step values are used to determine how the discrete space of these increments and decrements will be constructed.
An example of how it works is shown below:

In [9]:
env.reset()
print('CURRENT SETPOINTS VALUES: ', env.current_setpoints)
for i in range(5):
    action=env.action_space.sample()
    _,_,_,_,info=env.step(action)
    print('Action number ',i,': ',env.action_mapping[action])
    print('Setpoints update: ', info['action'])
env.close()

[2023-05-26 08:52:12,974] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-05-26 08:52:12,974] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-05-26 08:52:12,974] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-05-26 08:52:12,974] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-05-26 08:52:13,098] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/examples/Eplus-env-5Zone-hot-discrete-v1-res4/Eplus-env-sub_run1
[2023-05-26 08:52:13,098] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/examples/Eplus-env-5Zone-hot-discrete-v1-res4/Eplus-env-sub_run1
[2023-05-26 08:52:13,098] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus working di

/usr/local/lib/python3.10/dist-packages/opyplus/weather_data/weather_data.py:493: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  epw_content = self._headers_to_epw(use_datetimes=use_datetimes) + df.to_csv(


CURRENT SETPOINTS VALUES:  [21, 25]
Action number  0 :  [0.0, 1.5]
Setpoints update:  [21.0, 26.5]
Action number  1 :  [0.0, -0.5]
Setpoints update:  [21.0, 26.0]
Action number  2 :  [0.0, 2.0]
Setpoints update:  [21.0, 28.0]
Action number  3 :  [0.5, 0.0]
Setpoints update:  [21.5, 28.0]
Action number  4 :  [1.5, 0.0]
Setpoints update:  [22.5, 28.0]
[2023-05-26 08:52:19,821] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus simulation closed successfully. 
[2023-05-26 08:52:19,821] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus simulation closed successfully. 
[2023-05-26 08:52:19,821] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus simulation closed successfully. 
[2023-05-26 08:52:19,821] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus simulation closed successfully. 


## Normalization Wrapper

This wrapper is used to transform observation received from simulator in values between 0 and 1.
It is possible to define a list of variables you want to normalize, if you don't define this list, all environment
variables will be included. 

In order to normalize values, it is necessary a dictionary in order to store max and min values. 
Then, if a environment variable is not included in the dictionary specified in wrapper constructor, then the normalization 
for that variable will be skipped.

In [10]:
#Original env
env=gym.make('Eplus-5Zone-hot-discrete-v1')
env = NormalizeObservation(
        env=env,
        ranges=RANGES_5ZONE,
        variables=[
            'Zone Thermostat Heating Setpoint Temperature(SPACE1-1)',
            'Zone Thermostat Cooling Setpoint Temperature(SPACE1-1)',
            'Zone Air Temperature(SPACE1-1)'])

[2023-05-26 08:52:19,930] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model ExternalInterface object if it is not present...
[2023-05-26 08:52:19,930] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model ExternalInterface object if it is not present...
[2023-05-26 08:52:19,930] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model ExternalInterface object if it is not present...
[2023-05-26 08:52:19,930] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model ExternalInterface object if it is not present...
[2023-05-26 08:52:19,930] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model ExternalInterface object if it is not present...
[2023-05-26 08:52:19,931] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2023-05-26 08:52:19,931] EPLUS_ENV_5Zone-hot-discrete-v1_MainThre

We can see in the next code how the variables specified have been normalized correctly:

In [11]:
env.reset()
obs,_,_,_,_=env.step(env.action_space.sample())
obs_dict=dict(zip(env.variables['observation'],obs))
env.close()
print('OBSERVATION WITH NORMALIZATION IN VARIABLES SELECTED: ',obs_dict)

[2023-05-26 08:52:19,998] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-05-26 08:52:19,998] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-05-26 08:52:19,998] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-05-26 08:52:19,998] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-05-26 08:52:19,998] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-05-26 08:52:20,129] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/examples/Eplus-env-5Zone-hot-discrete-v1-res5/Eplus-env-sub_run1
[2023-05-26 08:52:20,129] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/examples/Eplus-env-5Zone-hot-discrete-v1

/usr/local/lib/python3.10/dist-packages/opyplus/weather_data/weather_data.py:493: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  epw_content = self._headers_to_epw(use_datetimes=use_datetimes) + df.to_csv(


[2023-05-26 08:52:26,849] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus simulation closed successfully. 
[2023-05-26 08:52:26,849] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus simulation closed successfully. 
[2023-05-26 08:52:26,849] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus simulation closed successfully. 
[2023-05-26 08:52:26,849] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus simulation closed successfully. 
[2023-05-26 08:52:26,849] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus simulation closed successfully. 
OBSERVATION WITH NORMALIZATION IN VARIABLES SELECTED:  {'year': 1991.0, 'month': 1.0, 'day': 1.0, 'hour': 0.0, 'Site Outdoor Air Drybulb Temperature(Environment)': 4.4, 'Site Outdoor Air Relative Humidity(Environment)': 65.0, 'Site Wind Speed(Environment)': 3.875, 'Site Wind Direction(Environment)': 145.0, 'Site Diffuse Solar Radiation Rate per Area(Environment)': 0.0, 'Site Direct 

## Logger Wrapper

Wrapper for logging all interactions between agent and environment. Logger class can be selected
in the constructor if other type of logging is required. For more information about *Sinergym* Logger visit [Logger](https://ugr-sail.github.io/sinergym/compilation/main/pages/output.html#logger)

In [12]:
env=gym.make('Eplus-5Zone-hot-discrete-v1')
env=LoggerWrapper(env)


env.reset()    
terminated = False
current_month = 0
while not terminated:
    a = env.action_space.sample()
    _,_,terminated,_,_=env.step(a)
env.close()

[2023-05-26 08:52:26,955] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model ExternalInterface object if it is not present...
[2023-05-26 08:52:26,955] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model ExternalInterface object if it is not present...
[2023-05-26 08:52:26,955] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model ExternalInterface object if it is not present...
[2023-05-26 08:52:26,955] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model ExternalInterface object if it is not present...
[2023-05-26 08:52:26,955] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model ExternalInterface object if it is not present...
[2023-05-26 08:52:26,955] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model ExternalInterface object if it is not present...
[2023-05-26 08:52:26,957] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating 

/usr/local/lib/python3.10/dist-packages/opyplus/weather_data/weather_data.py:493: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  epw_content = self._headers_to_epw(use_datetimes=use_datetimes) + df.to_csv(


[2023-05-26 08:52:39,939] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus simulation closed successfully. 
[2023-05-26 08:52:39,939] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus simulation closed successfully. 
[2023-05-26 08:52:39,939] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus simulation closed successfully. 
[2023-05-26 08:52:39,939] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus simulation closed successfully. 
[2023-05-26 08:52:39,939] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus simulation closed successfully. 
[2023-05-26 08:52:39,939] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus simulation closed successfully. 


Now, you can see in Sinergym output folder that you will have available `progress.csv` file and `monitor.csv` files in each episode.

## Multi Observation Wrapper

This wrapper will stack observation received in a history queue (size can be customized).

In [13]:
#Original environment
env=gym.make('Eplus-5Zone-hot-discrete-v1')
obs, info=env.reset()
print('BEFORE MULTI OBSERVATION: ',obs)

#Multi Observation environment
env=MultiObsWrapper(env, n=5, flatten=True)
obs, info=env.reset()

[2023-05-26 08:52:40,055] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model ExternalInterface object if it is not present...
[2023-05-26 08:52:40,055] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model ExternalInterface object if it is not present...
[2023-05-26 08:52:40,055] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model ExternalInterface object if it is not present...
[2023-05-26 08:52:40,055] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model ExternalInterface object if it is not present...
[2023-05-26 08:52:40,055] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model ExternalInterface object if it is not present...
[2023-05-26 08:52:40,055] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model ExternalInterface object if it is not present...
[2023-05-26 08:52:40,055] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating 

/usr/local/lib/python3.10/dist-packages/opyplus/weather_data/weather_data.py:493: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  epw_content = self._headers_to_epw(use_datetimes=use_datetimes) + df.to_csv(


BEFORE MULTI OBSERVATION:  [1.9910000e+03 1.0000000e+00 1.0000000e+00 0.0000000e+00 4.0000000e+00
 6.9000000e+01 3.0999999e+00 1.3000000e+02 0.0000000e+00 0.0000000e+00
 2.1000000e+01 2.5000000e+01 2.1000031e+01 0.0000000e+00 2.3247053e+01
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 8.0801924e+03]
[2023-05-26 08:52:47,252] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus episode completed successfully. 
[2023-05-26 08:52:47,252] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus episode completed successfully. 
[2023-05-26 08:52:47,252] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus episode completed successfully. 
[2023-05-26 08:52:47,252] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus episode completed successfully. 
[2023-05-26 08:52:47,252] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus episode completed successfully. 
[2023-05-26 08:52:47,252] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT

In [14]:
print('AFTER MULTI OBSERVATION: ',obs)
env.close()

AFTER MULTI OBSERVATION:  [1.9910000e+03 1.0000000e+00 1.0000000e+00 0.0000000e+00 4.0000000e+00
 6.9000000e+01 3.0999999e+00 1.3000000e+02 0.0000000e+00 0.0000000e+00
 2.1000000e+01 2.5000000e+01 2.1000031e+01 0.0000000e+00 2.3247053e+01
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 8.0801924e+03
 1.9910000e+03 1.0000000e+00 1.0000000e+00 0.0000000e+00 4.0000000e+00
 6.9000000e+01 3.0999999e+00 1.3000000e+02 0.0000000e+00 0.0000000e+00
 2.1000000e+01 2.5000000e+01 2.1000031e+01 0.0000000e+00 2.3247053e+01
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 8.0801924e+03
 1.9910000e+03 1.0000000e+00 1.0000000e+00 0.0000000e+00 4.0000000e+00
 6.9000000e+01 3.0999999e+00 1.3000000e+02 0.0000000e+00 0.0000000e+00
 2.1000000e+01 2.5000000e+01 2.1000031e+01 0.0000000e+00 2.3247053e+01
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 8.0801924e+03
 1.9910000e+03 1.0000000e+00 1.0000000e+00 0.0000000e+00 4.0000000e+00
 6.9000000e+01 3.0999999e+00 1.3000000e+02 0.000000

## Nesting wrappers

All the wrappers listed in this notebook can be nested in order to combine functionalities added. However, **the order** in which these wrappers are nested is critical to their proper functionality. We strongly recommend that it be in the same order in which they appear here or in the documentation. In case you want to nest a subset of the total, it is not a problem as long as you continue respecting this order.
We see and example:

In [15]:
env = gym.make('Eplus-5Zone-hot-discrete-v1')
env = MultiObjectiveReward(
    env=env,
    reward_terms=[
        'reward_energy',
        'reward_comfort'])
env = PreviousObservationWrapper(env, previous_variables=[
    'Zone Thermostat Heating Setpoint Temperature(SPACE1-1)',
    'Zone Thermostat Cooling Setpoint Temperature(SPACE1-1)',
    'Zone Air Temperature(SPACE1-1)'])
env = DatetimeWrapper(env)
env = DiscreteIncrementalWrapper(
    env, max_values=[
        22.5, 30.0], min_values=[
        15.0, 22.5], delta_temp=2, step_temp=0.5)
env = NormalizeObservation(
    env=env,
    ranges=RANGES_5ZONE,
    variables=[
        'Zone Thermostat Heating Setpoint Temperature(SPACE1-1)',
        'Zone Thermostat Cooling Setpoint Temperature(SPACE1-1)',
        'Zone Air Temperature(SPACE1-1)',
        'Zone Thermostat Heating Setpoint Temperature(SPACE1-1)_previous',
        'Zone Thermostat Cooling Setpoint Temperature(SPACE1-1)_previous',
        'Zone Air Temperature(SPACE1-1)_previous'])
env = LoggerWrapper(env=env, flag=True)
env = MultiObsWrapper(env=env, n=5, flatten=True)

[2023-05-26 08:52:54,713] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model ExternalInterface object if it is not present...
[2023-05-26 08:52:54,713] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model ExternalInterface object if it is not present...
[2023-05-26 08:52:54,713] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model ExternalInterface object if it is not present...
[2023-05-26 08:52:54,713] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model ExternalInterface object if it is not present...
[2023-05-26 08:52:54,713] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model ExternalInterface object if it is not present...
[2023-05-26 08:52:54,713] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model ExternalInterface object if it is not present...
[2023-05-26 08:52:54,713] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating 

Now we just simply use the environment with the wrappers, for example:

In [16]:
for i in range(1):
    obs, info = env.reset()
    terminated = False
    current_month = 0
    while not terminated:
        a = env.action_space.sample()
        obs, reward, terminated, truncated, info = env.step(a)
        if info['month'] != current_month:  # display results every month
            current_month = info['month']
            print('Reward: ', reward, info)
env.close()

[2023-05-26 08:52:54,789] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-05-26 08:52:54,789] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-05-26 08:52:54,789] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-05-26 08:52:54,789] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-05-26 08:52:54,789] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-05-26 08:52:54,789] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-05-26 08:52:54,789] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-05-26 08:52:54,789] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...


/usr/local/lib/python3.10/dist-packages/opyplus/weather_data/weather_data.py:493: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  epw_content = self._headers_to_epw(use_datetimes=use_datetimes) + df.to_csv(


Reward:  [-1.01287517197652, -0.0] {'timestep': 1, 'time_elapsed': 900, 'year': 1991, 'month': 1, 'day': 1, 'hour': 0, 'action': [22.0, 25.0], 'reward': -0.50643758598826, 'reward_energy': -1.01287517197652, 'reward_comfort': -0.0, 'total_energy': 10128.7517197652, 'abs_comfort': 0.0, 'temperatures': [21.39952294733769]}
Reward:  [-0.5713704399231108, -0.0] {'timestep': 2976, 'time_elapsed': 2678400, 'year': 1991, 'month': 2, 'day': 1, 'hour': 0, 'action': [22.0, 26.0], 'reward': -0.2856852199615554, 'reward_energy': -0.5713704399231108, 'reward_comfort': -0.0, 'total_energy': 5713.704399231107, 'abs_comfort': 0.0, 'temperatures': [22.00053854409991]}
Reward:  [-0.0228454128515769, -0.70775714157962] {'timestep': 5664, 'time_elapsed': 5097600, 'year': 1991, 'month': 3, 'day': 1, 'hour': 0, 'action': [15.0, 26.0], 'reward': -0.36530127721559846, 'reward_energy': -0.0228454128515769, 'reward_comfort': -0.70775714157962, 'total_energy': 228.454128515769, 'abs_comfort': 0.70775714157962, '